## Deployment of Diagnosis Model

In [3]:
#import libraries
import keras
import tensorflow as tf
from keras.preprocessing import image
import json
import requests
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Conv2D, MaxPooling2D, GlobalMaxPooling2D, BatchNormalization
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from keras.metrics import top_k_categorical_accuracy
from keras.models import load_model
from keras.optimizers import Adam

In [2]:
#need to downgrade to 2.2.0 at the moment so that the model accuracy is the same
#saving model doesn't seem to work on tf ver. 2.3.1

#!pip install 'tensorflow == 2.2.0'
print(tf.__version__)

2.2.0


## Saving Model

In [3]:
#load model 4b
model_4b = load_model('../model/diagnosis_classifier/4b_smote_model_cnn.h5') 

In [4]:
#loading our 4b model's best weights
model_4b.load_weights("../weights/diagnosis/weights-improvement-24-0.76.hdf5")

In [ ]:
#save version 1 of model 4b
model_4b.save('../deployment/diagnosis/1')

Deployment of the model is done through Tensorflow Serving on Docker. Installation of Docker is required.

### Importing image and its label to put into model

In [4]:
images = []
img = image.load_img('../datasets/processed_image/ISIC_0024356.jpg', target_size=(284, 284))
img_array = image.img_to_array(img)
img = img_array/255
images.append(img)
x = np.array(images)

In [5]:
#load in labeled_ground_truth
testing_labels = pd.read_csv('../datasets/deployment/testing_labels.csv')

In [6]:
#create list of columns to be excluded from target variables y
columns_dropped = [x for x in testing_labels.columns if x not in ['benign', 'malignant', 'precancerous']]

#create y that contains all 3 target variables/classes
y = np.array(testing_labels.drop(columns = columns_dropped))

In [7]:
y_true = np.argmax(y, axis=1)
y_true

array([0])

### Running the server

In [8]:
data = json.dumps({"instances": x.tolist()})
class_names = ['benign', 'malignant', 'precancerous']
headers = {"content-type": "application/json"}

json_response = requests.post('http://localhost:8501/v1/models/diagnosis:predict',
                              data=data,
                              headers=headers)

predictions = json.loads(json_response.text)['predictions']
print(predictions)


print('The model predicted this was a {} (class {})'.format(class_names[np.argmax(predictions)], np.argmax(predictions)))
print('It was actually {} (class {})'.format(class_names[int(y_true)], int(y_true)))

[[0.912082434, 0.0831930488, 0.00472460641]]
The model predicted this was a benign (class 0)
It was actually benign (class 0)
